### TASK:
To create a simulation of a container terminal using SimPy

### Components:
  - Vessels: Arrive at the terminal with an inter-arrival time following an exponential distribution. Each carries 150 containers.
  - Berths: Limited to 2 slots where vessels can dock.
  - Quay Cranes: 2 cranes are available to unload containers from vessels to trucks. Each operation takes 3 minutes per container.
  - Trucks: 3 trucks transport containers from cranes to the yard, with a round trip taking 6 minutes.
  
### Requirements:  
  - Simulation Events: Logging for vessel arrival, berthing, container unloading, and truck movements.



In [184]:
import simpy
import random
import itertools

In [185]:

NUM_CONTAINERS_PER_VESSEL = 150 # Each vessel holds 150 containers
VESSEL_INTERVAL = 300           # Mean time between arrivals in minutes
CRANE_UNLOADING_TIME = 3        # Time to unload one container in minutes
TRUCK_TRANSPORT_TIME = 6        # Round trip transport time for a truck in minutes
SIM_TIME = 3000                 # Total simulation time in minutes
RANDOM_SEED = 42


In [186]:
class Terminal:
    """
    Simulates a container terminal with specified unloading and transport times.
    """
    def __init__(self, env, unload_time, transport_time):
        self.env = env
        self.berths = simpy.Resource(env, 2)
        self.cranes = simpy.Resource(env, 2)
        self.trucks = simpy.Resource(env, 3)
        self.unload_time = unload_time
        self.truck_transport_time = transport_time

    def unload_vessel(self, vessel_name):
        """
        Simulates the unloading of containers from a vessel.
        """
        crane_request = self.cranes.request()
        yield crane_request  # Wait for a crane to become available
        
        # since only 1 crane is used by a vessel, all 150 containers will use the same crane_request
        for container_number in range(1, NUM_CONTAINERS_PER_VESSEL + 1):
            truck_request = self.trucks.request()
            yield truck_request  # Wait for a truck to become available
            
            print(f'{self.env.now:.0f} : {vessel_name} : Crane starts unloading container-{container_number} to truck')
            yield self.env.timeout(self.unload_time)
            
            print(f'{self.env.now:.0f} : {vessel_name} : Truck left with container-{container_number}')
            
            # Start the truck transport process without waiting for it to complete
            self.env.process(self.simulate_transport(vessel_name, container_number, truck_request))
            
        self.cranes.release(crane_request)  # Release the crane

    def simulate_transport(self, vessel_name, container_number, truck_request):
        """
        Simulates the truck's round trip for transporting a container.
        """
        yield self.env.timeout(self.truck_transport_time)
        print(f'{self.env.now:.0f} : {vessel_name} : Truck returned after transporting container-{container_number}')
        self.trucks.release(truck_request)  # Release the truck


def vessel_arrival(env, vessel_name, terminal):
    """
    Processes the arrival and unloading of a vessel at the terminal.
    """
    print(f'{env.now:.0f} : {vessel_name} : Arrived at terminal_ _ _ _ _ _<-\___/')
    with terminal.berths.request() as berth_request:
        yield berth_request  # Wait for berth availability
        print(f'{env.now:.0f} : {vessel_name} : Berthed')
        
        yield env.process(terminal.unload_vessel(vessel_name))
        print(f'{env.now:.0f} : {vessel_name} : Finished unloading and is leaving_ _ _ _ _ \___/->')


def vessel_generator(env, terminal):
    """
    Generates vessels arriving at the terminal.
    """
    vessel_count = itertools.count(1)
    while True:
        t = random.expovariate(1.0 / VESSEL_INTERVAL)
        yield env.timeout(t)
        env.process(vessel_arrival(env, f'Vessel {next(vessel_count)}', terminal))


print('Container Terminal Unload Simulation')
print('-----------------------------------------------------')
print('Time(m) | Vessel |  Action')
print('-----------------------------------------------------')
random.seed(RANDOM_SEED)

env = simpy.Environment()
terminal = Terminal(env, CRANE_UNLOADING_TIME, TRUCK_TRANSPORT_TIME)
env.process(vessel_generator(env, terminal))
env.run(until=SIM_TIME)


Container Terminal Unload Simulation
-----------------------------------------------------
Time(m) | Vessel |  Action
-----------------------------------------------------
306 : Vessel 1 : Arrived at terminal_ _ _ _ _ _<-\___/
306 : Vessel 1 : Berthed
306 : Vessel 1 : Crane starts unloading container-1 to truck
309 : Vessel 1 : Truck left with container-1
309 : Vessel 1 : Crane starts unloading container-2 to truck
312 : Vessel 1 : Truck left with container-2
312 : Vessel 1 : Crane starts unloading container-3 to truck
314 : Vessel 2 : Arrived at terminal_ _ _ _ _ _<-\___/
314 : Vessel 2 : Berthed
315 : Vessel 1 : Truck returned after transporting container-1
315 : Vessel 1 : Truck left with container-3
315 : Vessel 2 : Crane starts unloading container-1 to truck
318 : Vessel 1 : Truck returned after transporting container-2
318 : Vessel 2 : Truck left with container-1
318 : Vessel 1 : Crane starts unloading container-4 to truck
321 : Vessel 1 : Truck returned after transporting contai

2043 : Vessel 4 : Crane starts unloading container-139 to truck
2046 : Vessel 4 : Truck returned after transporting container-138
2046 : Vessel 4 : Truck left with container-139
2046 : Vessel 3 : Crane starts unloading container-142 to truck
2049 : Vessel 3 : Truck returned after transporting container-141
2049 : Vessel 3 : Truck left with container-142
2049 : Vessel 4 : Crane starts unloading container-140 to truck
2052 : Vessel 4 : Truck returned after transporting container-139
2052 : Vessel 4 : Truck left with container-140
2052 : Vessel 3 : Crane starts unloading container-143 to truck
2055 : Vessel 3 : Truck returned after transporting container-142
2055 : Vessel 3 : Truck left with container-143
2055 : Vessel 4 : Crane starts unloading container-141 to truck
2058 : Vessel 4 : Truck returned after transporting container-140
2058 : Vessel 4 : Truck left with container-141
2058 : Vessel 3 : Crane starts unloading container-144 to truck
2061 : Vessel 3 : Truck returned after transpo